In [ ]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt
from collections import Counter
import json
import typing as tp
import ast
from datetime import datetime
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [ ]:
leads_last = sorted(lib.find_tables_in_hahn_folder("//home/cloud_analytics/dwh/raw/crm/leads"))[-1]

In [ ]:
leads_last

# 1. Добавление

In [ ]:
req = f"""
SELECT
    task_id,
    first_name,
    last_name,
    account_name,
    phone_mobile,
    email,
    status,
    lead_source,
    lead_source_description,
    description,
    timezone,
    website,
    'korsakovakris, baranovamargo' as reserved_for
FROM (
    SELECT DISTINCT
        lead_id as task_id, 
        any(first_name) as first_name,
        any(last_name) as last_name,
        any(account_name) as account_name, 
        any(phone_mobile) as phone_mobile,
        any(email) as email,
        status,
        lead_source,
        any(lead_source_description) as lead_source_description,
        any(description) as description,
        any(timezone) as timezone
    from 
        "//home/cloud_analytics/kulaga/leads_cube"
    where lead_source = 'isv' 
    and status in ('New', 'Assigned')
    and lower(tag_name) = 'прозвон_кц'
    GROUP BY task_id, status, lead_source, tag_name
) as a
INNER JOIN (
    SELECT
        DISTINCT
        id as lead_id,
        max(lower(website)) as website
    FROM "{leads_last}"
    GROUP BY lead_id
) as b
ON a.task_id == b.lead_id
WHERE phone_mobile != ''
    AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
"""
df = lib.execute_query(req.encode('utf-8'))

In [ ]:
df.shape

In [ ]:
var_df = lib.execute_query(f"""
SELECT
    DISTINCT
        phone_mobile,
        email,
        website,
        lead_id
FROM "//home/cloud_analytics/kulaga/leads_cube" as a
LEFT JOIN (
    SELECT
        DISTINCT
        id as lead_id,
        max(lower(website)) as website
    FROM "{leads_last}"
    GROUP BY lead_id
) as b
ON a.lead_id == b.lead_id
WHERE lead_source = 'var'
AND phone_mobile != ''
AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
""")

In [ ]:
old_df = lib.execute_query("""
SELECT 
    DISTINCT
    task_id
FROM "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/isv_source_drop_task_id"
FORMAT TabSeparatedWithNames
""")
df = df[~df["task_id"].isin(old_df["task_id"])]

In [ ]:
var_df['phone_mobile'] = var_df['phone_mobile'].apply(lambda x: lib.works_with_phones(x))
df['phone_mobile'] = df['phone_mobile'].apply(lambda x: lib.works_with_phones(x))

repeat_var = df[(df['phone_mobile'].isin(var_df['phone_mobile'].fillna('trash'))) |
                (df['email'].isin(var_df['email'].fillna('trash'))) |
                (df['website'].isin(var_df['website'].fillna('trash')))]

In [ ]:
df = df[~df["task_id"].isin(repeat_var["task_id"])]

In [ ]:
if len(df) > 0:
    lib.save_table(str(datetime.now()).replace(" ", "T"), 
               "//home/cloud_analytics/call_center/input/isv", df)

## dropped list update

In [ ]:
added_df = pd.DataFrame(df["task_id"])

In [ ]:
lib.save_table("isv_source_drop_task_id", 
               "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output", added_df, append=True)

# 1.25 Дубликат Var

In [ ]:
double_isv_var_df = pd.DataFrame()
double_isv_var_df['CRM_Lead_ID'] = repeat_var['task_id']
double_isv_var_df['Tags'] = '["дубль заявки isv/var"]'
double_isv_var_df['Assigned_to'] = 'eknyaz'

In [ ]:
dt = lib.execute_query("""
SELECT
    toInt64(NOW()) as Timestamp
FORMAT TabSeparatedWithNames
""")['Timestamp'].iloc[0]


In [ ]:
double_isv_var_df['Timestamp'] = dt

In [ ]:
double_isv_var_df

In [ ]:
lib.save_table('update_leads', "//home/cloud_analytics/export/crm/update_call_center_leads",
               double_isv_var_df, append=True)

# 1.5 Повторное добавление

In [ ]:
second_df = lib.execute_query("""
select 
    task_id, 
    assignment_dt, 
    coalesce(billing_account_id, '') as billing_account_id,
    first_name,
    last_name,
    account_name, 
    phone_mobile,
    email,
    status,
    lead_source,
    lead_source_description,
    description,
    timezone,
    '' as reserved_for
from 
(select DISTINCT
    task_id,
    toDate(assignment_approve_time/1000) as assignment_dt
from
    "//home/cloud_analytics/call_center/output/isv_output") as t0 
left join 
(select 
    lead_id, 
    argMax(billing_account_id, date_modified) as billing_account_id,
    argMax(first_name, date_modified) as first_name,
    argMax(last_name, date_modified) as last_name,
    argMax(account_name, date_modified) as account_name, 
    argMax(phone_mobile, date_modified) as phone_mobile,
    argMax(email, date_modified) as email,
    argMax(status, date_modified) as status,
    argMax(lead_source, date_modified) as lead_source,
    argMax(lead_source_description, date_modified) as lead_source_description,
    'no billing_account_id' as description,
    argMax(timezone, date_modified) as timezone
from 
    "//home/cloud_analytics/kulaga/leads_cube"
group by 
    lead_id
) as t1 
on t0.task_id = t1.lead_id

where billing_account_id in (null, '')
and status != 'Recycled'
and (toDate(today()) - assignment_dt >= 3
OR task_id in (SELECT DISTINCT task_id FROM 
"//home/cloud_analytics/call_center/output/isv_output"
WHERE CAST(Tags as String) like '%non_resident%')
)
AND phone_mobile != ''
AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
""".encode('utf-8'))

In [ ]:
old_df = lib.execute_query("""
SELECT 
    DISTINCT
    task_id
FROM "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/isv_source_drop_task_id_second_call"
FORMAT TabSeparatedWithNames
""")
second_df = second_df[~second_df["task_id"].isin(old_df["task_id"])]

In [ ]:
if len(second_df) > 0:
    lib.save_table(str(datetime.now()).replace(" ", "T"), 
               "//home/cloud_analytics/call_center/input/isv", second_df)

In [ ]:
added_df = pd.DataFrame(second_df["task_id"])
lib.save_table("isv_source_drop_task_id_second_call", 
               "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output", added_df, append=True)

# 2. Update Lead

In [ ]:
req = """
SELECT
    toInt64(NOW()) as Timestamp,
    CAST(task_id as String) as CRM_Lead_ID,
    Billing_account_id as Billing_account_id_New,
    billing_account_id as Billing_account_id,
    relevant_contact_name as First_name,
    lower(relevant_contact_email) as Email_New,
    lower(email) as Email,
    relevant_contact_phone as Phone_2,
    multiIf(result == 'decline', 'Не готов общаться',
            status = 'no_response', 'Не дозвонились',
            null) as Dimensions,
    dimension as raw_dimensions,
    
     multiIf(CAST(Tags as String) like '%принять%', 'validated',
            CAST(Tags as String) like '%под вопросом%', 'tentative',
            CAST(Tags as String) like '%non_resident%', 'non_resident',
            CAST(Tags as String) like '%отклонить%', 'annulated', 
            null) as Tags,
    if(instruction_required == 1, 'need_ba', null) as Tags_New,
    if (Tags == 'annulated', 'Recycled', null) as Status,
    'eknyaz' as Assigned_to,
    toDateTime(toInt64(assignment_approve_time / 1000)) as Last_communication_date,

    toString(ifNull(Comment, '')) || 
        '; Собственные продукты: ' || toString(ifNull(own_product, '')) || 
        '; Аудитория/Профиль партнера:  ' || toString(ifNull(profile, '')) || 
        '; Модель тарификации: ' || toString(ifNull(tariffing, '')) || 
        '; ИТ-ресурсы: ' || toString(ifNull(it_resources, '')) ||  
        '; Рассматривает переход в Я.О: ' || toString(ifNull(has_interest, 0))  || 
        '; Является платящим клиентом Я.О: ' || toString(ifNull(is_paid, 0))  || 
        '; Нужна инструкция по созданию биллинг аккаунта: ' || toString(ifNull(instruction_required, 0)) ||
        '; website: ' || toString(ifNull(website, '')) as Notes
FROM "//home/cloud_analytics/call_center/output/isv_output" as a
ANY LEFT JOIN (
    SELECT
        max(email) as email,
        max(billing_account_id) as billing_account_id,
        lead_id
    FROM "//home/cloud_analytics/kulaga/leads_cube"
    GROUP BY lead_id
) as b
ON CRM_Lead_ID == lead_id
FORMAT TabSeparatedWithNames
""".encode('utf-8')

update_df = lib.execute_query(req)

In [ ]:
def raw_yson_str_parser_for_russian_text(exm):
    need_del=False
    final_str = ""
    for sym in exm:
        if need_del:
            if len(re.sub('[^а-яА-Я]', '', sym)) != 0:
                need_del = False
            pass
        if not need_del and sym not in '[]':
            final_str += sym
        if sym in '[;]':
            need_del=True
    if len(final_str) > 0 and final_str[-1] == ';':
        final_str = final_str[:-1]
    return final_str.split(';')

In [ ]:
update_df['raw_dimensions'] = update_df['raw_dimensions'].apply(lambda x:
                                                           raw_yson_str_parser_for_russian_text(x)
                                                            if not pd.isnull(x) else x)


In [ ]:
def concat_dimensions(row):
    arr = []
    if not pd.isnull(row['Dimensions']) and row['Dimensions'] != '':
        arr.append(row['Dimensions'])
    if isinstance(row['raw_dimensions'], list):
        if '' in row['raw_dimensions']:
            row['raw_dimensions'].remove('')
        arr += row['raw_dimensions']
    return arr

In [ ]:
update_df["Dimensions"] = update_df.apply(lambda row: concat_dimensions(row), axis=1)

In [ ]:
def concat_array_strings_old_new(x, name):
    add = []
    if not pd.isnull(x[name]):
        if name.lower() == 'email':
            add.append(lib.works_with_emails(x[name].strip()))
        else:
            add.append(x[name].strip())
    if not pd.isnull(x[name + "_New"]):
        if name.lower() == 'email':
            add.append(lib.works_with_emails(x[name + "_New"].strip()))
        else:
            add.append(x[name + "_New"].strip())
    if len(list(set(add))) > 0:
        return list(set(add))
    return np.nan

In [ ]:
update_df['Email'] = update_df.apply(lambda x: concat_array_strings_old_new(x, "Email"), axis=1)
update_df['Billing_account_id'] = update_df.apply(lambda x: 
                                                  concat_array_strings_old_new(x, "Billing_account_id"), axis=1)
update_df['Tags'] = update_df.apply(lambda x: concat_array_strings_old_new(x, "Tags"), axis=1)

In [ ]:
arr_cols = ['Email', 'Billing_account_id', 'Tags', 'Dimensions']
for col in arr_cols:
    update_df[col] = update_df[col].astype(str)
    update_df[col] = update_df[col].apply(lambda x: x.replace("'", '"') if not pd.isnull(x) else np.nan)

In [ ]:
update_df.drop(columns = ['raw_dimensions'], inplace=True)

In [ ]:
dropped = []
for col in update_df.columns:
    if "_New" in col:
        dropped.append(col)
update_df.drop(columns=dropped, inplace=True)

In [ ]:
old_df = lib.execute_query("""
SELECT
    DISTINCT 
    CRM_Lead_ID || '__' || toString(ifNull(Last_communication_date, '')) as key
FROM "//home/cloud_analytics/export/crm/update_call_center_leads/update_leads"
FORMAT TabSeparatedWithNames
""")

In [ ]:
update_df = update_df[~update_df[['CRM_Lead_ID', 'Last_communication_date']]\
          .apply(lambda x: x['CRM_Lead_ID'] + "__" + x['Last_communication_date'], axis=1)\
          .isin(old_df['key'])]

In [ ]:
update_df['Phone_2'] = update_df['Phone_2'].astype(str)

In [ ]:
update_df.replace("nan", np.nan, inplace=True)

In [ ]:
lib.save_table('update_leads', "//home/cloud_analytics/export/crm/update_call_center_leads",
               update_df, append=True)